## Introduction to Iceberg Architecture

#### Launching a Spark Session with Iceberg

In [1]:
import cml.data_v1 as cmldata

CONNECTION_NAME = "go01-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco
Hive Session ID = ba8f8f3e-1900-41c3-b249-edf7f74b0743


+--------------------+
|           namespace|
+--------------------+
|         01_car_data|
|           01_car_dw|
|      adash_car_data|
|             airline|
|          airline_dw|
|            airlines|
|        airlines_csv|
|       airlines_csv1|
|   airlines_csv_vish|
|    airlines_iceberg|
|   airlines_iceberg1|
|airlines_iceberg_...|
|      airlines_mjain|
|          airquality|
|          atlas_demo|
|            bankdemo|
|              bhagan|
|             cdedemo|
|        cdp_overview|
|        cgsifacebook|
+--------------------+
only showing top 20 rows



In [2]:
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.app.startTime', '1682526095537'),
 ('spark.repl.local.jars',
  'file:///runtime-addons/spark320-17-hf1-6xa3lk/opt/spark/optional-lib/iceberg-spark-runtime-3.2_2.12-0.14.1.1.17.7215.0-31.jar'),
 ('spark.driver.bindAddress', '100.100.178.94'),
 ('spark.network.crypto.enabled', 'true'),
 ('spark.sql.hive.hwc.execution.mode', 'spark'),
 ('spark.kerberos.renewal.credentials', 'ccache'),
 ('spark.sql.catalog.spark_catalog',
  'org.apache.iceberg.spark.SparkSessionCatalog'),
 ('spark.dynamicAllocation.maxExecutors', '49'),
 ('spark.eventLog.dir', 'file:///sparkeventlogs'),
 ('spark.hadoop.yarn.resourcemanager.principal', 'pauldefusco'),
 ('spark.kubernetes.driver.annotation.cluster-autoscaler.kubernetes.io/safe-to-evict',
  'false'),
 ('spark.ui.port', '20049'),
 ('spark.yarn.access.hadoopFileSystems',
  's3a://go01-demo/warehouse/tablespace/external/hive'),
 ('spark.sql.extensions',
  'com.qubole.spark.hiveacid.HiveAcidAutoConvertExtension,org.ap

### Iceberg Architecture

![alt text](../img/iceberg-metadata.png)

#### Iceberg Catalog

Iceberg comes with catalogs that enable SQL commands to manage tables and load them by name. Catalogs are configured using properties under spark.sql.catalog.(catalog_name).

In [3]:
# Show catalog and database
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|  default|
+-------------+---------+



In [4]:
# Create a new database
spark.sql("CREATE DATABASE IF NOT EXISTS spark_catalog.lakehouse_catalog")
spark.sql("USE spark_catalog.lakehouse_catalog")

DataFrame[]

In [5]:
# Show catalog and database
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+-----------------+
|      catalog|        namespace|
+-------------+-----------------+
|spark_catalog|lakehouse_catalog|
+-------------+-----------------+



#### Create an Iceberg Table with Spark SQL

In [6]:
spark.sql("DROP TABLE IF EXISTS customers_table")

DataFrame[]

In [7]:
spark.sql("CREATE TABLE IF NOT EXISTS customers_table (id BIGINT, state STRING, country STRING, dob TIMESTAMP) USING iceberg PARTITIONED BY ( hours(dob))")

DataFrame[]

#### Verify that a Metadata JSON file has been created under the Metadata directory

In [9]:
metadata_path = "s3a://go01-demo/warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata"

In [10]:
import boto3

s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

for object_summary in my_bucket.objects.filter(Prefix="warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata"):
    print(object_summary.key)
    metadata_file = object_summary.key

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/00000-e19a6100-50ef-4556-a1ad-d13018d6286e.metadata.json


In [11]:
import pandas as pd
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file).toPandas()

,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-updated-ms,location,metadata-log,partition-spec,partition-specs,properties,schema,schemas,snapshot-log,snapshots,sort-orders,table-uuid
0,0,-1,0,0,1,4,1000,1682526144303,s3a://go01-demo/warehouse/tablespace/external/...,[],"[(1000, dob_hour, 4, hour)]","[([Row(field-id=1000, name='dob_hour', source-...","(pauldefusco,)","([(1, id, False, long), (2, state, False, stri...","[([Row(id=1, name='id', required=False, type='...",[],[],"[([], 0)]",24ac86bb-a78d-49f8-9c8a-6288f0991a25


![alt text](../img/s3_metadata.png)

#### Notice that no snapshots or other files have been created as data has not yet been inserted.

In [12]:
spark.sql("SELECT * FROM lakehouse_catalog.customers_table.history").show()

+---------------+-----------+---------+-------------------+
|made_current_at|snapshot_id|parent_id|is_current_ancestor|
+---------------+-----------+---------+-------------------+
+---------------+-----------+---------+-------------------+



In [13]:
spark.sql("SELECT * FROM lakehouse_catalog.customers_table.snapshots;").show()

+------------+-----------+---------+---------+-------------+-------+
|committed_at|snapshot_id|parent_id|operation|manifest_list|summary|
+------------+-----------+---------+---------+-------------+-------+
+------------+-----------+---------+---------+-------------+-------+



In [14]:
spark.sql("SELECT * FROM lakehouse_catalog.customers_table.files;").show()

+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
|content|file_path|file_format|spec_id|partition|record_count|file_size_in_bytes|column_sizes|value_counts|null_value_counts|nan_value_counts|lower_bounds|upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+



In [15]:
spark.sql("SELECT * FROM lakehouse_catalog.customers_table.manifests;").show()

+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+
|content|path|length|partition_spec_id|added_snapshot_id|added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries|
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+



In [16]:
spark.sql("SELECT * FROM lakehouse_catalog.customers_table.all_data_files;").show()

+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
|content|file_path|file_format|spec_id|partition|record_count|file_size_in_bytes|column_sizes|value_counts|null_value_counts|nan_value_counts|lower_bounds|upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+
+-------+---------+-----------+-------+---------+------------+------------------+------------+------------+-----------------+----------------+------------+------------+------------+-------------+------------+-------------+



In [17]:
spark.sql("SELECT * FROM lakehouse_catalog.customers_table.all_manifests;").show()

+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+---------------------+
|content|path|length|partition_spec_id|added_snapshot_id|added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries|reference_snapshot_id|
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+---------------------+
+-------+----+------+-----------------+-----------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+-------------------+-------

### Table Insert

In [18]:
from pyspark.sql.functions import date_format

In [19]:
spark.sql("INSERT INTO lakehouse_catalog.customers_table VALUES (1, 'CA', 'USA', cast(date_format('2000-01-01 00:00:00', 'yyyy-MM-dd HH:mm:ss') as timestamp))")

DataFrame[]

#### Validate that data has been added to the data folder

In [20]:
QUERY = "select h.made_current_at,\
            s.operation,\
            h.snapshot_id,\
            h.is_current_ancestor,\
            s.summary['spark.app.id']\
        from lakehouse_catalog.customers_table.history h\
        join lakehouse_catalog.customers_table.snapshots s\
            on h.snapshot_id = s.snapshot_id\
            order by made_current_at;"

In [21]:
spark.sql(QUERY).toPandas()

,made_current_at,operation,snapshot_id,is_current_ancestor,summary[spark.app.id]
0,2023-04-26 16:23:44.985,append,4069727943432154358,True,spark-application-1682526097424


![alt text](../img/s3_data_1.png)

![alt text](../img/s3_data_2.png)

#### Show all of the table’s data files and each file’s metadata.

#### Notice there are now two json files and two avro files. 

The second json file reflects the new table version after the insert. Now, a new table read operation will point to this new file and ignore the previous one.

The avro file with the "snap" prefix is the manifest list. The other avro file created is the corresponding manifest file.

In [25]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

metadata_file_list = []

for object_summary in my_bucket.objects.filter(Prefix="warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata"):
    print(object_summary.key +"\n")
    metadata_file_list.append(object_summary.key)

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/00000-e19a6100-50ef-4556-a1ad-d13018d6286e.metadata.json

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/00001-d9221d02-b558-473b-a7bf-347c90b86a9b.metadata.json

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/66be1815-abfc-43c5-bf95-59720a6f3c79-m0.avro

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/snap-4069727943432154358-1-66be1815-abfc-43c5-bf95-59720a6f3c79.avro



In [26]:
metadata_file_list

['warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/00000-e19a6100-50ef-4556-a1ad-d13018d6286e.metadata.json',
 'warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/00001-d9221d02-b558-473b-a7bf-347c90b86a9b.metadata.json',
 'warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/66be1815-abfc-43c5-bf95-59720a6f3c79-m0.avro',
 'warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/snap-4069727943432154358-1-66be1815-abfc-43c5-bf95-59720a6f3c79.avro']

In [34]:
import pandas as pd

print("Showing " + metadata_file_list[0])
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[0]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/00000-e19a6100-50ef-4556-a1ad-d13018d6286e.metadata.json


,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-updated-ms,location,metadata-log,partition-spec,partition-specs,properties,schema,schemas,snapshot-log,snapshots,sort-orders,table-uuid
0,0,-1,0,0,1,4,1000,1682526144303,s3a://go01-demo/warehouse/tablespace/external/...,[],"[(1000, dob_hour, 4, hour)]","[([Row(field-id=1000, name='dob_hour', source-...","(pauldefusco,)","([(1, id, False, long), (2, state, False, stri...","[([Row(id=1, name='id', required=False, type='...",[],[],"[([], 0)]",24ac86bb-a78d-49f8-9c8a-6288f0991a25


In [35]:
print("Showing " + metadata_file_list[1])
spark.read.option("multiline","true").json("s3a://go01-demo/" + metadata_file_list[1]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/00001-d9221d02-b558-473b-a7bf-347c90b86a9b.metadata.json


,current-schema-id,current-snapshot-id,default-sort-order-id,default-spec-id,format-version,last-column-id,last-partition-id,last-updated-ms,location,metadata-log,partition-spec,partition-specs,properties,refs,schema,schemas,snapshot-log,snapshots,sort-orders,table-uuid
0,0,4069727943432154358,0,0,1,4,1000,1682526224985,s3a://go01-demo/warehouse/tablespace/external/...,[(s3a://go01-demo/warehouse/tablespace/externa...,"[(1000, dob_hour, 4, hour)]","[([Row(field-id=1000, name='dob_hour', source-...","(pauldefusco,)","((4069727943432154358, branch),)","([(1, id, False, long), (2, state, False, stri...","[([Row(id=1, name='id', required=False, type='...","[(4069727943432154358, 1682526224985)]",[(s3a://go01-demo/warehouse/tablespace/externa...,"[([], 0)]",24ac86bb-a78d-49f8-9c8a-6288f0991a25


In [36]:
print("Showing " + metadata_file_list[2])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[2]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/66be1815-abfc-43c5-bf95-59720a6f3c79-m0.avro


,status,snapshot_id,data_file
0,1,4069727943432154358,(s3a://go01-demo/warehouse/tablespace/external...


In [37]:
print("Showing " + metadata_file_list[3])
spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[3]).toPandas()

Showing warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/snap-4069727943432154358-1-66be1815-abfc-43c5-bf95-59720a6f3c79.avro


,manifest_path,manifest_length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,partitions,added_rows_count,existing_rows_count,deleted_rows_count
0,s3a://go01-demo/warehouse/tablespace/external/...,6189,0,4069727943432154358,1,0,0,"[(False, False, [56, 3, 4, 0], [56, 3, 4, 0])]",1,0,0


In [46]:
file3 = spark.read.format("avro").load("s3a://go01-demo/" + metadata_file_list[3]).toPandas()

In [49]:
file3['manifest_path'][0]

's3a://go01-demo/warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/66be1815-abfc-43c5-bf95-59720a6f3c79-m0.avro'

### Table Update

Create a staging table

In [38]:
spark.sql("DROP TABLE IF EXISTS lakehouse_catalog.staging")

DataFrame[]

In [39]:
spark.sql("CREATE TABLE IF NOT EXISTS lakehouse_catalog.staging\
            (id BIGINT, state STRING, country STRING, dob TIMESTAMP)\
            USING iceberg\
            PARTITIONED BY ( hours(dob))")

DataFrame[]

In [40]:
spark.sql("INSERT INTO lakehouse_catalog.staging\
            VALUES (1, 'ID', 'USA', cast(date_format('2000-01-01 00:00:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (2, 'CA', 'USA', cast(date_format('2000-01-03 00:10:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (3, 'AZ', 'USA', cast(date_format('2000-01-04 00:01:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (4, 'NV', 'USA', cast(date_format('2000-01-02 00:02:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (5, 'OR', 'USA', cast(date_format('2000-01-03 00:03:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (10, 'WA', 'USA', cast(date_format('2000-01-04 00:03:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (3, 'UT', 'USA', cast(date_format('2000-01-01 00:04:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (11, 'CO', 'USA', cast(date_format('2000-01-01 00:03:00', 'yyyy-MM-dd HH:mm:ss') as timestamp)),\
            (6, 'CO', 'USA', cast(date_format('2000-01-01 00:03:00', 'yyyy-MM-dd HH:mm:ss') as timestamp))")

DataFrame[]

Merge Into Customers Table

In [41]:
spark.sql("MERGE INTO lakehouse_catalog.customers_table\
            USING (SELECT * FROM lakehouse_catalog.staging) a\
            ON customers_table.id = a.id\
            WHEN MATCHED THEN UPDATE SET customers_table.state = a.state\
            WHEN NOT MATCHED THEN INSERT *")

DataFrame[]

In [42]:
spark.sql("SELECT * FROM lakehouse_catalog.customers_table.snapshots;").toPandas()

,committed_at,snapshot_id,parent_id,operation,manifest_list,summary
0,2023-04-26 16:23:44.985,4069727943432154358,NaN,append,s3a://go01-demo/warehouse/tablespace/external/...,{'spark.app.id': 'spark-application-1682526097...
1,2023-04-26 16:44:55.770,314145575071817500,4.069728e+18,overwrite,s3a://go01-demo/warehouse/tablespace/external/...,"{'added-data-files': '4', 'total-equality-dele..."


In [43]:
spark.sql("SELECT * FROM lakehouse_catalog.customers_table.manifests;").toPandas()

,content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,0,s3a://go01-demo/warehouse/tablespace/external/...,6365,0,314145575071817500,4,0,0,0,0,0,"[(False, False, 2000-01-01-00, 2000-01-04-00)]"
1,0,s3a://go01-demo/warehouse/tablespace/external/...,6191,0,314145575071817500,0,0,1,0,0,0,"[(False, False, 2000-01-01-00, 2000-01-01-00)]"


In [44]:
spark.sql("SELECT * FROM lakehouse_catalog.customers_table.all_data_files;").toPandas()

,content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id
0,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(262968,)",4,1277,"{1: 57, 2: 74, 3: 61, 4: 80}","{1: 4, 2: 4, 3: 4, 4: 4}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [67, 79], 3: ...","{1: [11, 0, 0, 0, 0, 0, 0, 0], 2: [85, 84], 3:...",None,[4],None,0
1,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(262992,)",1,1130,"{1: 39, 2: 37, 3: 38, 4: 39}","{1: 1, 2: 1, 3: 1, 4: 1}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: [4, 0, 0, 0, 0, 0, 0, 0], 2: [78, 86], 3: ...","{1: [4, 0, 0, 0, 0, 0, 0, 0], 2: [78, 86], 3: ...",None,[4],None,0
2,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(263016,)",2,1176,"{1: 46, 2: 43, 3: 61, 4: 47}","{1: 2, 2: 2, 3: 2, 4: 2}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: [2, 0, 0, 0, 0, 0, 0, 0], 2: [67, 65], 3: ...","{1: [5, 0, 0, 0, 0, 0, 0, 0], 2: [79, 82], 3: ...",None,[4],None,0
3,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(263040,)",2,1177,"{1: 47, 2: 43, 3: 61, 4: 47}","{1: 2, 2: 2, 3: 2, 4: 2}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: [3, 0, 0, 0, 0, 0, 0, 0], 2: [65, 90], 3: ...","{1: [10, 0, 0, 0, 0, 0, 0, 0], 2: [87, 65], 3:...",None,[4],None,0
4,0,s3a://go01-demo/warehouse/tablespace/external/...,PARQUET,0,"(262968,)",1,1106,"{1: 33, 2: 31, 3: 32, 4: 39}","{1: 1, 2: 1, 3: 1, 4: 1}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [67, 65], 3: ...","{1: [1, 0, 0, 0, 0, 0, 0, 0], 2: [67, 65], 3: ...",None,[4],None,0


#### There is a new metadata file (json) prefixed by 0002

#### There is a new manifest list file (avro) prefixed by "snap"

#### There is a new manifest file (avro)

In [60]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket("go01-demo")

for object_summary in my_bucket.objects.filter(Prefix="warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata"):
    print(object_summary.key +"\n")
    metadata_file = object_summary.key

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/00000-2e11b1f4-cdb2-4cfb-a3e7-1d572c5186ef.metadata.json

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/00001-aa607d9c-52c9-4208-bd50-8a29103a0269.metadata.json

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/00002-8a99b449-0dd8-4dd1-bfb0-53c510c12334.metadata.json

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/dde17409-0dcd-42f0-a2d5-8c8838b49f7b-m0.avro

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/fdf81d84-1f96-4bda-a83c-a98a3a35805d-m0.avro

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/fdf81d84-1f96-4bda-a83c-a98a3a35805d-m1.avro

warehouse/tablespace/external/hive/lakehouse_catalog.db/customers_table/metadata/snap-2397883159392124377-1-fdf81d84-1f96-4bda-a83c-a98a3a35805d.avro

warehouse/tablespace/external/hive/lakehouse_catalo

### Time Travel 

In [81]:
snapshots_df = spark.sql("SELECT * FROM lakehouse_catalog.customers_table.snapshots;")

In [82]:
first_snapshot = snapshots_df.select("snapshot_id").head(1)[0][0]

#### Validate that the output dataframe only includes one row

In [85]:
spark.read\
    .option("snapshot-id", first_snapshot)\
    .format("iceberg")\
    .load("lakehouse_catalog.customers_table").toPandas()

,id,state,country,dob
0,1,CA,USA,2000-01-01


### Drop the Table

In [50]:
spark.sql("DROP TABLE IF EXISTS lakehouse_catalog.staging")

DataFrame[]